# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,9.06,97,0,0.45,ZA,1722657951
1,1,bethel,41.3712,-73.4140,24.83,90,100,1.54,US,1722657952
2,2,adamstown,-25.0660,-130.1015,20.31,78,8,4.22,PN,1722657952
3,3,anadyr,64.7500,177.4833,10.28,93,100,6.00,RU,1722657952
4,4,carnarvon,-24.8667,113.6333,20.51,55,0,6.46,AU,1722657952


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
#%%capture --no-display

# Configure the map plot
city_map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    color='City',
    alpha= 0.6,
    s='Humidity',
    scale= 0.6
)
# Display the map
city_map_plot_1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temperature_min = 20
ideal_temperature_max = 35
max_humidity = 60

filtered_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= ideal_temperature_min) &
    (city_data_df['Max Temp'] <= ideal_temperature_max) &
    (city_data_df['Country'] !='US') &
    (city_data_df['Country'] !='JP') &
    (city_data_df['Country'] !='KP') &
    (city_data_df['Country'] !='TH') &
    (city_data_df['Country'] !='VN') &
    (city_data_df['Humidity'] <= max_humidity)
]
print(len(filtered_df))

# Drop any rows with null values
filtered_df.dropna()

# Display sample data
filtered_df.head()

98


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,carnarvon,-24.8667,113.6333,20.51,55,0,6.46,AU,1722657952
17,17,yakutsk,62.0339,129.7331,20.98,28,0,3.00,RU,1722657956
36,36,tshikapa,-6.4167,20.8000,20.49,29,15,0.98,CD,1722657961
41,41,al bawiti,28.3492,28.8659,25.42,55,0,2.74,EG,1722657961
44,44,parauapebas,-6.0675,-49.9022,22.69,45,10,1.07,BR,1722657962


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.iloc[:,[1,8,2,3,5,]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
4,carnarvon,AU,-24.8667,113.6333,55,
17,yakutsk,RU,62.0339,129.7331,28,
36,tshikapa,CD,-6.4167,20.8000,29,
41,al bawiti,EG,28.3492,28.8659,55,
44,parauapebas,BR,-6.0675,-49.9022,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
yakutsk - nearest hotel: Орто Дойду
tshikapa - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
parauapebas - nearest hotel: Hotel Igarapé
klyuchi - nearest hotel: No hotel found
demre - nearest hotel: No hotel found
zhumysker - nearest hotel: No hotel found
qarazhal - nearest hotel: No hotel found
vorgashor - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
botou - nearest hotel: No hotel found
istgah-e rah ahan-e garmsar - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
yelizovo - nearest hotel: Николина Усадьба
cabo san lucas - nearest hotel: Comfort Rooms
georgetown - nearest hotel: Page 63 hostel
harqalah - nearest hotel: No hotel found
argo - nearest hotel: Apollon Hotel
kuching - nearest hotel: Hotel Samudera
al kharijah - nearest hotel: Kharga Hotel
yerofey pavlovich - nearest hotel: No hotel found
gigant - nearest hotel: No hotel found
aykhal - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
4,carnarvon,AU,-24.8667,113.6333,55,No hotel found
17,yakutsk,RU,62.0339,129.7331,28,Орто Дойду
36,tshikapa,CD,-6.4167,20.8000,29,No hotel found
41,al bawiti,EG,28.3492,28.8659,55,Old Oasis Hotel
44,parauapebas,BR,-6.0675,-49.9022,45,Hotel Igarapé
...,...,...,...,...,...,...
558,zheleznogorsk-ilimskiy,RU,56.5768,104.1217,40,Магнетит
559,shimanovsk,RU,52.0000,127.7000,33,No hotel found
560,pemangkat,ID,1.1667,108.9667,60,No hotel found
570,campo maior,BR,-4.8278,-42.1686,49,Hotel Pousada do Lago


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
#%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=600,
    frame_height=400,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)